<a href="https://colab.research.google.com/github/shahriariit/COVID-19-Xray/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2

import os

In [0]:
from keras.applications.resnet import ResNet50
from keras.models import Sequential
from keras.layers import Dense,Dropout

### 
### Below systax is available with TensorFlow 1.11 onwards but this upgrade is not available for Kaggle kernel yet
###
#import tensorflow as tf

#print(tf.__version__)
#import tensorflow as tf
#from tf.keras.applications import ResNet50
#from tf.keras.models import Sequential

In [0]:
import cv2
import os

TRAIN_DATADIR= r'C:\Users\Asus\Documents\corona\XRay\covid-chestxray-dataset\data\train'
#CATEGORIES =  ["COVID_19","NORMAL","Viral_Pneumonia"]
CATEGORIES =  ["COVID19","NORMAL","PNEUMONIA"]

IMG_SIZE = 100
TEST_DATADIR = r'C:\Users\Asus\Documents\corona\XRay\covid-chestxray-dataset\data\test'
training_data = []
testing_data= []
new_array = []

def create_training_data():
    
    for category in CATEGORIES:
        train_path = os.path.join(TRAIN_DATADIR, category)
        test_path = os.path.join(TEST_DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(train_path):
            try:
                img_array =cv2.imread(os.path.join(train_path,img))

                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                new_array = new_array.flatten()
              
                training_data.append([new_array,class_num])
            except Exception as e :
                pass
        
        for img in os.listdir(test_path):
            try:
                img_array2 =cv2.imread(os.path.join(test_path,img))

                new_array2 = cv2.resize(img_array2, (IMG_SIZE, IMG_SIZE))
                new_array2 = new_array2.flatten()
              
                training_data.append([new_array2,class_num])
            except Exception as e :
                pass
                
create_training_data()           
print(len(training_data))
print(len(testing_data))



In [0]:
import random 
random.shuffle(training_data)
random.shuffle(testing_data)

#for sample in training_data[:10]:
 #   print(sample[1])
X=[]
y=[]


for features, label in training_data:
    X.append(features) 
    y.append(label) 

X = np.array(X).reshape(len(training_data),100,100,3)  
y =np.array(y)  
print(X.shape)  
print(y.shape)  
 


In [0]:
from keras.utils import np_utils
from keras.utils import to_categorical

y = to_categorical(y)
y.shape


In [0]:
print(X.shape)
print(y.shape)

In [0]:
#Still not talking about our train/test data or any pre-processing.
def Transfer():
    model = Sequential()

    # 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
    # NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
    model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
    model.add(Dropout(0.4))
    # 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
    model.add(Dense(3, activation = 'softmax'))

    # Say not to train first layer (ResNet) model as it is already trained
    model.layers[0].trainable = False
    sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [0]:
model = Transfer()
model.summary()

In [0]:
from keras import optimizers

from keras.applications.resnet50 import preprocess_input
preprocess_input(X) 
preprocess_input(y) 

In [0]:
#set early stopping criteria
from keras.callbacks import ModelCheckpoint, EarlyStopping
avg_accuracy = []
avg_loss=[]
avg_validation_accuracy=[]
avg_validation_loss=[]
pat = 3 #this is the number of epochs with no improvment after which the training will stop
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

#define the model checkpoint callback -> this will keep on saving the model as a physical file
model_checkpoint = ModelCheckpoint('fas_mnist_1.h5', verbose=1, save_best_only=True)

#define a function to fit the model
def fit_and_evaluate(t_x, val_x, t_y, val_y, EPOCHS=20, BATCH_SIZE=128):
    model = None
    model = Transfer() 
    results = model.fit(t_x, t_y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping, model_checkpoint], 
              verbose=1, validation_split=0.1) 
    
   # avg_train_accuracy.append((results.history['accuracy'][-1])*100)
    #avg_train_loss.append((results.history['loss'][-1])*100)
    #avg_validation_accuracy.append((results.history['val_accuracy'][-1])*100)
    #avg_validation_loss.append((results.history['val_loss'][-1])*100)
    print('Covid19 Train acc: {}'.format(np.round((results.history['accuracy'][-1])*100)))  
    model.load_weights('fas_mnist_1.h5')
     
    #print("Val Score: ", model.evaluate(val_x, val_y))
    scores = model.evaluate(val_x, val_y)
    print("val Scores: ",scores)
    #print(model.metrics_names[0]+" :"+scores[0]) 
    #print(model.metrics_names[1]+" :"+scores[1]*100) 
    avg_accuracy.append(scores[1] * 100) 
    avg_loss.append(scores[0]) 
    return results 

In [0]:

from sklearn.model_selection import train_test_split
n_folds= 3
epochs= 5
batch_size= 16

#save the model history in a list after fitting so that we can plot later
model_history = [] 

for i in range(n_folds):
    print("Training on Fold: ",i+1)
    t_x, val_x, t_y, val_y = train_test_split(X, y, test_size=0.3, 
                                               random_state = np.random.randint(1,1000, 1)[0]) 
    model_history.append(fit_and_evaluate(t_x, val_x, t_y, val_y, epochs, batch_size)) 
    print("======="*12, end="\n\n\n") 

In [0]:
avg_acc = np.mean(avg_accuracy)
avg_loss = np.mean(avg_loss)
print(avg_acc)
print(avg_loss)

In [0]:
plt.title('Train Accuracy vs Val Accuracy')
 
plt.plot(model_history[0].history['accuracy'], label='Training Accuracy Fold 1',color='black' )
plt.plot(model_history[0].history['val_accuracy'], label='Val Accuracy Fold 1', color='black', linestyle = "dashdot")
plt.plot(model_history[1].history['accuracy'], label='Training Accuracy Fold 2',color='red')

plt.plot(model_history[1].history['val_accuracy'], label='Val Accuracy Fold 2', color='red', linestyle = "dashdot")
plt.plot(model_history[2].history['accuracy'], label='Training Accuracy Fold 3',color='green')

plt.plot(model_history[2].history['val_accuracy'], label='Val Accuracy Fold 3', color='green', linestyle = "dashdot")



plt.legend()
plt.show()

In [0]:
plt.title('Val loss')
 
plt.plot(model_history[0].history['val_loss'], label='Val Accuracy Fold 1', color='black', linestyle = "dashdot")

plt.plot(model_history[1].history['val_loss'], label='Val Accuracy Fold 2', color='red', linestyle = "dashdot")

plt.plot(model_history[2].history['val_loss'], label='Val Accuracy Fold 3', color='yellow', linestyle = "dashdot")




plt.legend()
plt.show()